# Split Apply Combine

Often times, we will want to split a dataset based on a key value in a column, and do something within this group, and return this information for each group.  We will often see this referred to as the **split-apply-combine** paradigm.  With Pandas, we will often use the `groupby` method to carry out the splitting piece of this. 

**OBJECTIVES**:

- Use `groupby` to split dataset into groups, and groups within groups
- Apply different methods to `groupby` objects
- Use aggregation methods on `groupby` objects
- Write and use functions of our own on `groupby` objects with `apply`

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

In [2]:
df = pd.DataFrame({'street': ['a', 'a', 'b', 'b', 'a'],
                  'agent': ['one', 'one', 'two', 'one', 'two'],
                  'sq_ft': np.random.randint(1000, 4500, 5),
                  'price': np.random.randint(100000, 600000, 5)})

In [3]:
df.head()

,street,agent,sq_ft,price
0,a,one,2149,371767
1,a,one,3122,539163
2,b,two,4446,548206
3,b,one,3293,380489
4,a,two,3915,308952


### `groupby` Basics

To begin, it is important to recognize that the result of using the `groupby` method is a `groupby` object.  This is an object that has simply grouped the data according to our input.  From here, we can select columns like usual and apply basic quantitative methods.  If we wanted to know the average square footage by agent in our mini-data above, we select the column we are concerned with (`df['sq_ft']`), and group this by agent(`.groupby(df['agent'])`).  We are returned an object for which we can then apply the mean to.  Below, we save our grouped data as `grouped`, and can subsequently perform a variety of methods.

In [4]:
df['sq_ft'].groupby(df['agent'])

In [10]:
grouped = df['sq_ft'].groupby(df['agent'])

In [11]:
grouped.head()

0    2149
1    3122
2    4446
3    3293
4    3915
Name: sq_ft, dtype: int64

In [6]:
grouped.mean()

agent
one    2854.666667
two    4180.500000
Name: sq_ft, dtype: float64

In [7]:
grouped.count()

agent
one    3
two    2
Name: sq_ft, dtype: int64

In [8]:
grouped.describe()

,count,mean,std,min,25%,50%,75%,max
agent,,,,,,,,
one,3.0,2854.666667,617.077251,2149.0,2635.50,3122.0,3207.50,3293.0
two,2.0,4180.500000,375.473701,3915.0,4047.75,4180.5,4313.25,4446.0


We can pass more than one value to group by, and will be returned an object that has two levels of indices.  For example, if we wanted to know:

$$
\textit{What is the average price for each agent by street?}
$$

We will select the price column, group the data by agent and street, and apply the mean method to this.  Notice that there are two levels of indices, *agent* and *street*.  We can return a dataframe that unstacks the levels with the `unstack` method.  

In [12]:
means = df['price'].groupby([df['agent'], df['street']]).mean()

In [13]:
means

agent  street
one    a         455465
       b         380489
two    a         308952
       b         548206
Name: price, dtype: int64

In [14]:
means.keys()

MultiIndex(levels=[['one', 'two'], ['a', 'b']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['agent', 'street'])

In [15]:
means.unstack()

street,a,b
agent,,
one,455465,380489
two,308952,548206


In [16]:
means[0]

455465

In [17]:
means[3]

548206

In [18]:
means.shape

(4,)

### Example I 

Using our tips dataset example, we can explore a few straightforward questions using `groupby()`.  

1. What is the average tip by gender?
2. What is the max tip on each day?
3. What is the min tip on each day by time?
4. What is the average bill by party size?

In [19]:
tips = sns.load_dataset('tips')

In [20]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [35]:
tips.groupby('sex')['tip'].mean()

sex
Male      3.089618
Female    2.833448
Name: tip, dtype: float64

In [43]:
tips.groupby('sex')[['tip']].mean()
#tips.groupby(['sex'])[['tip']].mean()

,tip
sex,
Male,3.089618
Female,2.833448


In [65]:
tips.groupby('day')[['tip']].max()

,tip
day,
Thur,6.70
Fri,4.73
Sat,10.00
Sun,6.50


In [66]:
#tips.groupby(['day', 'time'])['tip'].min().sort_values(ascending = True)
tips.groupby(['size', 'day'])[[ 'tip']].mean().unstack()

tip                              
day     Thur       Fri       Sat       Sun
size                                      
1     1.8300  1.920000  1.000000       NaN
2     2.4425  2.644375  2.517547  2.816923
3     2.6925  3.000000  3.797778  3.120667
4     4.2180  4.730000  4.123846  4.087778
5     5.0000       NaN  3.000000  4.046667
6     5.3000       NaN       NaN  5.000000

In [56]:
tips.groupby(['size', 'day'])[[ 'tip']].mean().unstack()

tip                              
day     Thur       Fri       Sat       Sun
size                                      
1     1.8300  1.920000  1.000000       NaN
2     2.4425  2.644375  2.517547  2.816923
3     2.6925  3.000000  3.797778  3.120667
4     4.2180  4.730000  4.123846  4.087778
5     5.0000       NaN  3.000000  4.046667
6     5.3000       NaN       NaN  5.000000

In [52]:
tips.groupby(['size', 'day'])[[ 'tip']].mean()

tip
size day           
1    Thur  1.830000
     Fri   1.920000
     Sat   1.000000
     Sun        NaN
2    Thur  2.442500
     Fri   2.644375
     Sat   2.517547
     Sun   2.816923
3    Thur  2.692500
     Fri   3.000000
     Sat   3.797778
     Sun   3.120667
4    Thur  4.218000
     Fri   4.730000
     Sat   4.123846
     Sun   4.087778
5    Thur  5.000000
     Fri        NaN
     Sat   3.000000
     Sun   4.046667
6    Thur  5.300000
     Fri        NaN
     Sat        NaN
     Sun   5.000000

Average bill and tip by smoker and time.

In [67]:
tips.groupby(['smoker', 'time'])[['total_bill', 'tip']].mean()

total_bill       tip
smoker time                        
Yes    Lunch    17.399130  2.834348
       Dinner   21.859429  3.066000
No     Lunch    17.050889  2.673778
       Dinner   20.095660  3.126887

In [70]:
tips.groupby(['smoker', 'time'])[['total_bill', 'tip']].mean().unstack()

total_bill                  tip          
time        Lunch     Dinner     Lunch    Dinner
smoker                                          
Yes     17.399130  21.859429  2.834348  3.066000
No      17.050889  20.095660  2.673778  3.126887

In [71]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


The results of the groupby object are iterable.  For example, if we group the tips data by smoker, we get back the group and the elements of this group.  For multiple indicies of groups, we pass these as tuples.

In [73]:
for name, group in tips.groupby(['smoker']):
    print(name)
    print(group)

Yes
     total_bill   tip     sex smoker   day    time  size
56        38.01  3.00    Male    Yes   Sat  Dinner     4
58        11.24  1.76    Male    Yes   Sat  Dinner     2
60        20.29  3.21    Male    Yes   Sat  Dinner     2
61        13.81  2.00    Male    Yes   Sat  Dinner     2
62        11.02  1.98    Male    Yes   Sat  Dinner     2
63        18.29  3.76    Male    Yes   Sat  Dinner     4
67         3.07  1.00  Female    Yes   Sat  Dinner     1
69        15.01  2.09    Male    Yes   Sat  Dinner     2
72        26.86  3.14  Female    Yes   Sat  Dinner     2
73        25.28  5.00  Female    Yes   Sat  Dinner     2
76        17.92  3.08    Male    Yes   Sat  Dinner     2
80        19.44  3.00    Male    Yes  Thur   Lunch     2
83        32.68  5.00    Male    Yes  Thur   Lunch     2
90        28.97  3.00    Male    Yes   Fri  Dinner     2
92         5.75  1.00  Female    Yes   Fri  Dinner     2
93        16.32  4.30  Female    Yes   Fri  Dinner     2
95        40.17  4.73    Ma

In [72]:
for (n1, n2), group in tips.groupby(['sex', 'smoker']):
    print(n1, n2)
    print(group)

Male Yes
     total_bill    tip   sex smoker   day    time  size
56        38.01   3.00  Male    Yes   Sat  Dinner     4
58        11.24   1.76  Male    Yes   Sat  Dinner     2
60        20.29   3.21  Male    Yes   Sat  Dinner     2
61        13.81   2.00  Male    Yes   Sat  Dinner     2
62        11.02   1.98  Male    Yes   Sat  Dinner     2
63        18.29   3.76  Male    Yes   Sat  Dinner     4
69        15.01   2.09  Male    Yes   Sat  Dinner     2
76        17.92   3.08  Male    Yes   Sat  Dinner     2
80        19.44   3.00  Male    Yes  Thur   Lunch     2
83        32.68   5.00  Male    Yes  Thur   Lunch     2
90        28.97   3.00  Male    Yes   Fri  Dinner     2
95        40.17   4.73  Male    Yes   Fri  Dinner     4
96        27.28   4.00  Male    Yes   Fri  Dinner     2
97        12.03   1.50  Male    Yes   Fri  Dinner     2
98        21.01   3.00  Male    Yes   Fri  Dinner     2
105       15.36   1.64  Male    Yes   Sat  Dinner     2
106       20.49   4.06  Male    Yes   S

In [74]:
#one-liner to create dictionary of groups
pieces = dict(list(tips.groupby('sex')))

In [75]:
pieces['Male'].head()

,total_bill,tip,sex,smoker,day,time,size
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2.00,Male,No,Sun,Dinner,2


In [76]:
pieces['Female'].head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
11,35.26,5.00,Female,No,Sun,Dinner,4
14,14.83,3.02,Female,No,Sun,Dinner,2
16,10.33,1.67,Female,No,Sun,Dinner,3


### Data Aggregation

Here, we are interested in using the aggreate function to apply functions that we have developed which return scalar values.  For example, we can write a function that determines the range of tips on a given day.  We are taking all of the values for tips by group, and finding a single value for this.  Similarly, we could do something like investigate the quantiles of each group.

In [77]:
grouped = tips.groupby('day')

In [78]:
def ranger(df):
    return df.max() - df.min()

In [81]:
grouped.agg(ranger)

,total_bill,tip,size
day,,,
Thur,35.60,5.45,5
Fri,34.42,3.73,3
Sat,47.74,9.00,4
Sun,40.92,5.49,4


In [82]:
grouped.quantile(0.7)

0.7,size,tip,total_bill
day,,,
Thur,2.0,3.000,18.990
Fri,2.0,3.150,19.134
Sat,3.0,3.164,22.470
Sun,3.0,3.695,24.570


In [83]:
grouped.quantile(0.9)['tip']

day
Thur    4.920
Fri     4.060
Sat     4.802
Sun     5.035
Name: tip, dtype: float64

We can add a tip percentage column as follows.

In [114]:
tips['pct_tip'] = tips['tip']/tips['total_bill']

In [85]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,pct_tip
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [86]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg('mean')

sex     smoker
Male    Yes       0.152771
        No        0.160669
Female  Yes       0.182150
        No        0.156921
Name: pct_tip, dtype: float64

In [87]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(['mean', 'std', ranger])

mean       std    ranger
sex    smoker                              
Male   Yes     0.152771  0.090588  0.674707
       No      0.160669  0.041849  0.220186
Female Yes     0.182150  0.071595  0.360233
       No      0.156921  0.036421  0.195876

In [88]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg([('Average Tipping Percentage', 'mean'), ('Standard Deviation', 'std'), ('Range', ranger)])

Average Tipping Percentage  Standard Deviation     Range
sex    smoker                                                          
Male   Yes                       0.152771            0.090588  0.674707
       No                        0.160669            0.041849  0.220186
Female Yes                       0.182150            0.071595  0.360233
       No                        0.156921            0.036421  0.195876

In [89]:
funcs = ['mean', 'max', 'min', 'std']

In [90]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(funcs)

mean       max       min       std
sex    smoker                                        
Male   Yes     0.152771  0.710345  0.035638  0.090588
       No      0.160669  0.291990  0.071804  0.041849
Female Yes     0.182150  0.416667  0.056433  0.071595
       No      0.156921  0.252672  0.056797  0.036421

In [91]:
#overwrite results with new
#column names
func_named = [('Average', 'mean'), ('Maximum', 'max')]

In [92]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(func_named)

Average   Maximum
sex    smoker                    
Male   Yes     0.152771  0.710345
       No      0.160669  0.291990
Female Yes     0.182150  0.416667
       No      0.156921  0.252672

In [93]:
#pass different aggregation functions
tips.groupby(['sex', 'smoker']).agg({'tip': np.min, 'pct_tip': np.max})

tip   pct_tip
sex    smoker                
Male   Yes     1.00  0.710345
       No      1.25  0.291990
Female Yes     1.00  0.416667
       No      1.00  0.252672

In [94]:
#pass multiple aggregation functions
tips.groupby(['sex', 'smoker']).agg({'tip': [np.min, np.mean], 'pct_tip': np.max})

tip             pct_tip
               amin      mean      amax
sex    smoker                          
Male   Yes     1.00  3.051167  0.710345
       No      1.25  3.113402  0.291990
Female Yes     1.00  2.931515  0.416667
       No      1.00  2.773519  0.252672

In [ ]:
#ignore index labels
tips.groupby(['sex', 'smoker'], as_index=False).agg({'tip': [np.min, np.mean], 'pct_tip': np.max})

### Exercise

Using the titanic data, let's use the `groupby` function to provide aggregate information about groups.

1. How many male and female passengers survived and died?
2. How many passengers from each class lived and died?
3. Create a new column that uses the `age` column to create the groups:
  - Children
  - Teenagers
  - Young Adults
  - Middle Aged
  - Older
  - Senior
  
You are free to define these as you see fit.  Further, you should use better and easier labels in your code.  Investigate the survival within each of these groups.

In [95]:
titanic = pd.read_csv('data/eda_data/titanic.csv')

In [96]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
survived    891 non-null int64
pclass      891 non-null int64
name        891 non-null object
sex         891 non-null object
age         714 non-null float64
sibsp       891 non-null int64
parch       891 non-null int64
ticket      891 non-null object
fare        891 non-null float64
cabin       204 non-null object
embarked    889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 76.6+ KB


In [97]:
titanic.head()

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [106]:
# How many male and female passengers survived and died?
titanic.groupby(['sex','survived'])[['survived']].count().unstack()

survived     
survived        0    1
sex                   
female         81  233
male          468  109

In [117]:
#2. How many passengers from each class lived and died?
titanic.groupby(['pclass','survived'])[['survived']].count().unstack()

survived     
survived        0    1
pclass                
1              80  136
2              97   87
3             372  119

In [120]:
titanic['age_group'] = pd.cut(titanic.age, [12,20,40,60,80,110], labels = ['child', 'teenager', 'grown', 'older', 'senior'])

In [121]:
titanic.head()

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,age_group
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,teenager
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,teenager
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,teenager
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,teenager
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,teenager


In [ ]:
titanic.groupby(['sex', 'survived'])['survived'].count()

In [ ]:
titanic.groupby(['pclass', 'survived'])['survived'].count().unstack()

In [ ]:
age_group = []
for age in titanic.age:
    if age <= 12:
        age_group.append('child')
    elif age < 20:
        age_group.append('teenager')
    elif age < 50:
        age_group.append('mid-age')
    elif age < 70:
        age_group.append('older')
    else:
        age_group.append('old')

In [ ]:
titanic['age_group'] = age_group

In [ ]:
titanic.head()

In [ ]:
titanic.groupby(['age_group', 'survived'])['survived'].count()

In [ ]:
titanic.groupby(['age_group', 'survived'])['survived'].count().unstack()

### `apply`

Beyond aggregation functions, we can apply a more general call to functions that don't necessarily return a scalar value.  For example, suppose we wanted to pass a function that will take the top 5 tip percentages.  Then, we can apply this to different groupings of the data.  Because our function takes a column argument as well as a top number, we can call these using the `apply` method also.

In [123]:
def top(df, n=5, column='pct_tip'):
    return df.sort_values(by = column)[-n:]

In [124]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   pct_tip
smoker                                                                   
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=2, column = 'total_bill')

In [ ]:
tips.groupby('smoker', group_keys = False).apply(top)

In [ ]:
tips.groupby('smoker')['pct_tip'].describe(percentiles = [])

In [ ]:
tips.groupby('smoker')['pct_tip'].describe(percentiles = []).unstack()

### Example for filling `na`

Suppose we have a dataset that we want to use group specific values as replacements for missing values.  In the example below, it seems we have some missing bidderrate values.  Perhaps we make the assumption that an acceptable replacement would be the average bidder rating for each auction.  To apply this, we first split the data and pass an anonymous `lambda` function that will be applied to each group.  In this case, we are filling the `na` values with the `mean`.

In [125]:
auction = pd.read_csv('data/auction/auction.csv')

In [126]:
auction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10681 entries, 0 to 10680
Data columns (total 9 columns):
auctionid       10681 non-null int64
bid             10681 non-null float64
bidtime         10681 non-null float64
bidder          10665 non-null object
bidderrate      10670 non-null float64
openbid         10681 non-null float64
price           10681 non-null float64
item            10681 non-null object
auction_type    10681 non-null object
dtypes: float64(5), int64(1), object(3)
memory usage: 751.1+ KB


In [127]:
auction.bidderrate.describe()

count    10670.000000
mean        31.936739
std        120.594074
min         -4.000000
25%          1.000000
50%          5.000000
75%         21.000000
max       3140.000000
Name: bidderrate, dtype: float64

In [128]:
auction.groupby('auctionid').mean().head()

,bid,bidtime,bidderrate,openbid,price
auctionid,,,,,
1638843936,1167.857143,4.378416,38.428571,500.0,1625.0
1638844284,362.500000,1.244913,16.500000,200.0,500.0
1638844464,554.236250,6.225163,9.125000,300.0,740.0
1638844729,284.545455,6.895609,29.636364,225.0,320.0
1638893549,144.500000,2.588555,1.600000,99.0,177.5


In [ ]:
fill_bidrate = lambda g: g.fillna(g.mean())

In [ ]:
auction_filled = pd.merge(auction, auction.groupby('auctionid').apply(fill_bidrate))

In [ ]:
auction_filled.head()

### Further Reading

- [Pandas `groupby` documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html)
- [Datacamp Article](https://www.datacamp.com/community/tutorials/pandas-split-apply-combine-groupby)